[View in Colaboratory](https://colab.research.google.com/github/aminzabardast/Tensorflow-Tutorials/blob/dev/3_Simple_FC_Network.ipynb)

# A Simple Fully Connected Artificial Neural Network

In [0]:
print('Hello World!')

Hello World!
